In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm
import cv2
import time
from skimage.feature import peak_local_max

import keras2onnx
import onnxruntime as rt
import onnx

from sklearn.cluster import KMeans

In [6]:
## help to plot all the layers
def plt_multi(img,m,bin_den,max_draw, nchw):
    fig=plt.figure(figsize=(50, 15))
    columns = 4
    rows = 2
    if nchw:
        m_plot = np.array(tf.transpose(m[bin_den], [0, 2, 3, 1]))
    else:
        m_plot = m[bin_den]
    for i in range(0, rows*columns-1):
        fig.add_subplot(rows, columns, i+1).set_title(i)
        img_plot = img
        if max_draw > 0:
            try:
    #             coordinates = peak_local_max(m_plot[0, :, :, i], min_distance=34, threshold_abs = 0.7)
    #             print(coordinates)
    #             print(coordinates[:, 1].shape[0])
                _,ly = cv2.threshold(m_plot[0, :, :, i],max_draw,1, cv2.THRESH_BINARY)
                ly = np.transpose(np.nonzero(ly))
                kmeans = KMeans(n_clusters=2, random_state=0).fit(ly)
                coordinates = kmeans.cluster_centers_
    #             print(i)
    #             print(coordinates)

                plt.scatter(coordinates[:, 1], coordinates[:, 0],s=5, c='r')
                plt.imshow(img)
                plt.imshow(m_plot[0, :, :, i], alpha=0.5,cmap=cm.binary)

            except:
                plt.imshow(m_plot[0, :, :, i],cmap=cm.binary)

        else:
            plt.imshow(img)
            plt.imshow(m_plot[0, :, :, i], alpha=0.5,cmap=cm.binary)
        
        
    fig.add_subplot(rows, columns, 8).set_title("img")
    plt.imshow(img,cmap=cm.binary)
    plt.show()

In [8]:
# MIMED
model = tf.keras.models.load_model('/tf/project/instrument-tracking/src/saved_model/MIMED_rd34_sig10_09_28_12_47_aug0.5_decay_default.h5', compile=False)

# ENDOVIS
# model = tf.keras.models.load_model('/tf/project/instrument-pose-tracking/src/tf_data/checkpoint/aug_proba_0.5_decay_default_model_rd34_sigma10_skip.ckpt', compile=False)

In [ ]:
# MIMED
img = cv2.imread('/tf/project/test/MIMED_test_img_0.png')
img = cv2.resize(img,(320, 256))
img_nchw = tf.transpose(img, [2, 0, 1])
img_tf = np.array([img_nchw])


# ENDOVIS
# img = cv2.imread('/tf/project/test/ENDOVIS_test_img_0.png')
# img_nhwc = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img_tf = np.array([img_nhwc])

# for matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
m = model.predict(img_tf)

In [ ]:
# MIMED
plt_multi(img_rgb,m,1,0.7,1)


# ENDOVIS
# plt_multi(img_rgb,m,1,0.5,0)

In [ ]:
onnx_model = keras2onnx.convert_keras(model, model.name)

In [ ]:
content = onnx_model.SerializeToString()
sess = rt.InferenceSession(content)
input_name = sess.get_inputs()[0].name

In [ ]:
# MIMED
img_onnx = [img_nchw]


# ENDOVIS
# img_onnx = [img_nhwc]

In [ ]:
m = sess.run(None, {input_name: img_onnx})

In [ ]:
# MIMED
plt_multi(img_rgb,m,1,0.5,1)


# ENDOVIS
# plt_multi(img_rgb,m,1,0,0)

In [12]:
# temp_model_file = 'MIMED_model.onnx'
# keras2onnx.save_model(onnx_model, temp_model_file)